In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import route_dynamics.route_elevation.base_df as base
import geopandas as gpd

from shapely.geometry import mapping

In [3]:
routes_shp = '../data/rt' + str(22) + '_pts2.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

In [4]:
dat1 = pd.read_csv("../data/Trip183.csv", low_memory=False)
dat2 = pd.read_csv("../data/Zon183Unsum.csv")

In [5]:
trip183 = dat1[['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats',
                'Period', 'AnnRides']]

In [6]:
trip183unsum = dat2[['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs']]

In [7]:
period = 'PM'
route = 22
df = trip183unsum
df = df.drop(df[(df.Period != period)].index)
#df = df.drop(df[(df.InOut != direction)].index)
df = df.drop(df[(df.Route != route)].index)
df

,Route,Dir,Trip_ID,InOut,STOP_SEQ,STOP_ID,Period,AveOn,AveOff,AveLd,Obs
67506,22,N,39169396,I,52.0,19790,PM,0.3,0.8,10.2,38
67508,22,N,39169399,I,52.0,19790,PM,0.5,0.1,2.0,36
67509,22,N,39169398,I,52.0,19790,PM,0.4,0.3,3.4,35
67520,22,N,39169396,I,53.0,19810,PM,0.1,0.5,9.7,38
67528,22,N,39169398,I,53.0,19810,PM,0.0,0.0,3.4,35
...,...,...,...,...,...,...,...,...,...,...,...
68745,22,S,39169401,O,27.0,39980,PM,0.6,0.1,3.1,38
68752,22,S,39169397,O,28.0,39990,PM,2.0,0.5,11.4,37
68753,22,S,39169400,O,28.0,39990,PM,0.8,0.6,3.6,38
68754,22,S,39169401,O,28.0,39990,PM,1.4,0.2,4.2,38


In [8]:
final_df = df.sort_values(by=['InOut','Trip_ID', 'STOP_SEQ', 'STOP_ID'])
final_df

,Route,Dir,Trip_ID,InOut,STOP_SEQ,STOP_ID,Period,AveOn,AveOff,AveLd,Obs
67564,22,N,39169394,I,56.0,19861,PM,0.0,0.6,2.6,37
67930,22,N,39169394,I,57.0,36390,PM,0.0,2.5,0.1,37
67583,22,N,39169396,I,1.0,22264,PM,0.3,0.0,10.1,38
67591,22,N,39169396,I,2.0,22280,PM,0.2,0.7,9.6,38
67611,22,N,39169396,I,3.0,22290,PM,0.0,0.3,9.3,38
...,...,...,...,...,...,...,...,...,...,...,...
68745,22,S,39169401,O,27.0,39980,PM,0.6,0.1,3.1,38
68754,22,S,39169401,O,28.0,39990,PM,1.4,0.2,4.2,38
68389,22,S,39169401,O,29.0,22260,PM,0.2,0.1,4.3,38
68400,22,S,39169401,O,30.0,22261,PM,0.0,0.1,4.2,38


In [9]:
final_df = final_df[['InOut','STOP_SEQ', 'STOP_ID', 'AveLd']]
final_df = final_df.drop_duplicates(subset=['InOut','STOP_SEQ'], keep='first')
final_df

,InOut,STOP_SEQ,STOP_ID,AveLd
67564,I,56.0,19861,2.6
67930,I,57.0,36390,0.1
67583,I,1.0,22264,10.1
67591,I,2.0,22280,9.6
67611,I,3.0,22290,9.3
...,...,...,...,...
68744,O,27.0,39980,9.3
68757,O,28.0,39990,10.9
68388,O,29.0,22260,10.1
68401,O,30.0,22261,10.0


In [10]:
final_df = final_df.sort_values(by=['InOut','STOP_SEQ'])
final_df.reset_index()

,index,InOut,STOP_SEQ,STOP_ID,AveLd
0,67583,I,1.0,22264,10.1
1,67591,I,2.0,22280,9.6
2,67611,I,3.0,22290,9.3
3,67623,I,4.0,22300,7.0
4,67643,I,5.0,22320,6.5
...,...,...,...,...,...
83,68744,O,27.0,39980,9.3
84,68757,O,28.0,39990,10.9
85,68388,O,29.0,22260,10.1
86,68401,O,30.0,22261,10.0


In [11]:
final_df['AveLd'].mean()

8.231818181818179

In [12]:
empty_bus = 14061

In [14]:
final_df['Pass_Mass'] = final_df['AveLd']*80
final_df['Total_Mass'] = final_df['Pass_Mass'] + empty_bus
#final_df = final_df[['InOut', 'STOP_SEQ', 'STOP_ID', 'AveLd', 'Pass_Load', 'Pass_Mass', 'Total_Mass', 'geometry']]
final_df

,InOut,STOP_SEQ,STOP_ID,AveLd,Pass_Mass,Total_Mass
67583,I,1.0,22264,10.1,808.0,14869.0
67591,I,2.0,22280,9.6,768.0,14829.0
67611,I,3.0,22290,9.3,744.0,14805.0
67623,I,4.0,22300,7.0,560.0,14621.0
67643,I,5.0,22320,6.5,520.0,14581.0
...,...,...,...,...,...,...
68744,O,27.0,39980,9.3,744.0,14805.0
68757,O,28.0,39990,10.9,872.0,14933.0
68388,O,29.0,22260,10.1,808.0,14869.0
68401,O,30.0,22261,10.0,800.0,14861.0


In [16]:
stop_ids = final_df['STOP_ID'].values
stop_ids

array([22264, 22280, 22290, 22300, 22320, 22330, 22340, 22360, 22370,
       22380, 22390, 22400, 22410, 22420, 22430, 22440, 22450, 22460,
       22470, 22480, 22490, 22500, 85203, 85205, 85210, 85220, 85230,
       31731, 39432, 39433, 39426, 39427, 39428, 39429, 39438, 39439,
       39440, 39441, 39442, 39443, 39445, 39446, 39447, 39448, 39450,
       39451, 39452, 39453, 39454, 39455, 39456, 19790, 19810, 19830,
       19850, 19861, 36390, 36390, 31818, 20050, 20070, 20090, 20110,
       39790, 39800, 39810, 39820, 39830, 39840, 39850, 39860, 39870,
       39880, 39890, 39900, 39910, 39920, 39930, 39940, 39955, 39965,
       39973, 39976, 39980, 39990, 22260, 22261, 22264])

In [17]:
stops_riders = pd.DataFrame(stop_ids, columns = ['STOP_ID'])
stops_riders.head()

,STOP_ID
0,22264
1,22280
2,22290
3,22300
4,22320


In [18]:
stops = gpd.read_file(stops_shp)
stops.head() 

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
0,1,142,0,YES,None,80761,225897,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Cncrt-earth,None,0,8414.0,POINT (-122.22345 47.46961)
1,2,142,0,YES,None,80762,312598,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Base plate,None,0,8415.0,POINT (-122.22346 47.46579)
2,3,142,0,YES,None,80763,226757,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Earth,Single,0,8416.0,POINT (-122.22378 47.44596)
3,4,142,0,NO,None,80764,312386,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Cncrt-earth,None,0,8417.0,POINT (-122.22404 47.44358)
4,5,142,0,YES,None,80765,226757,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Away,2in metal,Earth,None,0,8418.0,POINT (-122.22358 47.44595)


In [19]:
stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)

In [20]:
stops['ROUTE_LIST']

0        F 153
1          153
2          153
3          153
4          153
         ...  
28565      3 4
28566      3 4
28567      3 4
28568        0
28569        0
Name: ROUTE_LIST, Length: 28570, dtype: object

In [21]:
stops_list = pd.DataFrame(columns=stops.columns)
stops_list

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry


In [22]:
test = pd.DataFrame(stops.loc[1])

In [23]:
test.transpose()

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
1,2,142,0,YES,None,80762,312598,ACT,REG,RNT,...,Lind Ave SW,A1 <=2 rts,KCM,Toward,2in metal,Base plate,None,0,8415.0,POINT (-122.22345991745925 47.465790007896665)


In [24]:
for i in range(0, len(stops)):

    if str(22) in (stops['ROUTE_LIST'][i]):
        for x in stops['ROUTE_LIST'][i].split(' '):
            if str(22) == x:
                row = pd.DataFrame(stops.iloc[i])
                stops_list = pd.concat([stops_list,row.transpose()])
            else:
                pass
    else:
        pass

In [25]:
stops_list.head()

,OBJECTID,CHANGE_NUM,MINOR_CHAN,ACCESSIBIL,ACCESSORY_,STOP_ID,TRANS_LINK,STOP_STATU,STOP_TYPE,AUTH_CODE,...,ON_STREET_,ROUTESIGN,ROUTESIGN_,SIGN_MOUNT,SIGNPOST,SIGNPOST_A,SCHEDULE_H,NUM_SHELTE,GISOBJID,geometry
2112,4113,141,2,YES,None,22390,9631,ACT,REG,SEA,...,SW 106th St,A1 <=2 rts,KCM,Away,2in metal,Unknown,Single,0,1904.0,POINT (-122.38433195435942 47.50819512138902)
2113,4114,141,2,YES,None,22400,37266,ACT,REG,SEA,...,SW 106th St,A1 <=2 rts,KCM,Away,2in metal,Unknown,Single,0,1905.0,POINT (-122.38166026194457 47.508170979516095)
2114,4115,141,2,YES,None,22410,196931,ACT,REG,SEA,...,39th Ave SW,A1 <=2 rts,KCM,Toward,Light,Unknown,None,0,1906.0,POINT (-122.38159919025402 47.50652329388237)
2115,4116,141,2,NO,None,22420,197065,ACT,REG,SEA,...,Marine View Dr SW,A1 <=2 rts,KCM,Toward,2in metal,Unknown,Single,0,1907.0,POINT (-122.38061719289107 47.504660570508065)
2116,4117,141,2,LTD,None,22430,153386,ACT,REG,SEA,...,Marine View Dr SW,A1 <=2 rts,KCM,Toward,2in metal,Unknown,Single,0,1908.0,POINT (-122.38093900902744 47.50282367585952)


In [26]:
coord_list_full = stops_list.drop_duplicates('STOP_ID')
coord_list = coord_list_full[['STOP_ID', 'geometry']]
coord_list

,STOP_ID,geometry
2112,22390,POINT (-122.38433195435942 47.50819512138902)
2113,22400,POINT (-122.38166026194457 47.508170979516095)
2114,22410,POINT (-122.38159919025402 47.50652329388237)
2115,22420,POINT (-122.38061719289107 47.504660570508065)
2116,22430,POINT (-122.38093900902744 47.50282367585952)
...,...,...
14641,39990,POINT (-122.36464609969123 47.5212294612714)
15601,22260,POINT (-122.36805264851088 47.51745623636245)
15602,22261,POINT (-122.37117037450214 47.51749166724578)
15605,22264,POINT (-122.37689915659708 47.516854308421095)


In [27]:
coord_list.loc[coord_list['STOP_ID'] == 22390].geometry.values

array([<shapely.geometry.point.Point object at 0x14aaa95a0>], dtype=object)

In [28]:
filler = np.zeros(len(final_df))

In [29]:
final_df = final_df.assign(geometry = filler)

In [33]:
for i in range(len(final_df)):
    stopid = final_df['STOP_ID'].iloc[i]
    copy = coord_list.loc[coord_list['STOP_ID'] == stopid].geometry.values
    final_df['geometry'].iloc[i] = copy

In [34]:
final_df.head()

,InOut,STOP_SEQ,STOP_ID,AveLd,Pass_Mass,Total_Mass,geometry
67583,I,1.0,22264,10.1,808.0,14869.0,[POINT (-122.37689915659708 47.516854308421095)]
67591,I,2.0,22280,9.6,768.0,14829.0,[POINT (-122.37693810386416 47.51379557835211)]
67611,I,3.0,22290,9.3,744.0,14805.0,[POINT (-122.3792737907224 47.5138076827721)]
67623,I,4.0,22300,7.0,560.0,14621.0,[POINT (-122.38213977047363 47.51382265450292)]
67643,I,5.0,22320,6.5,520.0,14581.0,[POINT (-122.38475476992342 47.513836046877465)]


In [32]:
stop_ids = stops_list['STOP_ID'].values

In [ ]:
len(stop_ids)

In [ ]:
geometry = stops_list.geometry.values

In [ ]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
xy_df['STOP_ID'] = stops_list['STOP_ID'].values

In [ ]:
xy_df

In [ ]:
stops_riders.value_counts()

In [ ]:
pd.merge(stops_riders, xy_df, on = 'STOP_ID', how='')

In [ ]:
def route_ridership(period, direction, route):
    
    df = trip183unsum
    df = df.drop(df[(df.Period != period)].index)
    df = df.drop(df[(df.InOut != direction)].index)
    df = df.drop(df[(df.Route != route)].index)

    final_df = df.sort_values(by=['Trip_ID', 'STOP_SEQ', 'STOP_ID'])
    seq_id = final_df[['STOP_SEQ', 'STOP_ID']]
    seq_id2 = seq_id.drop_duplicates(subset=[,'STOP_SEQ'], keep='first')
    seq_id3 = seq_id2.sort_values(by='STOP_SEQ')
    stopid_dic = dict(zip(seq_id3.STOP_SEQ, seq_id3.STOP_ID))
    riders = final_df.pivot(index='STOP_SEQ', columns='Trip_ID', values='AveLd')

    keyfind = list(riders.columns)
    mass_bus = [trip_dict[x] for x in keyfind]
    mode = mode_mass = max(set(mass_bus), key=mass_bus.count)
    rider_columns = list(riders.columns)
    kept_columns = []

    for i in range(0, len(mass_bus)):
        if mass_bus[i] == mode:
            kept_columns.append(rider_columns[i])

    riders_interm = riders[kept_columns]
    riders_kept = pd.DataFrame((riders_interm.mean(axis=1)), columns=['Mean'])
    riders_kept.Mean*=80
    riders_kept.reset_index(inplace=True)
    riders_kept.replace({"STOP_SEQ": stopid_dic}, inplace=True)
    riders_kept.columns = ['STOP_ID', 'Mean']



    return final_df, riders_kept, mode_mass

In [ ]:
df_45, riders_45, mass_45 = route_ridership('AM', 'I', 45)

In [ ]:
riders_45

In [ ]:
def stop_coord(num, riders_num):
    route_num = num
    
    route = base.create_gdf(routes_shp, 6)
    #points = base.extract_point_df(route)
    stops = gpd.read_file(stops_shp)
    stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)
    
    stops_list = pd.DataFrame()
    for i in range(0, len(stops)):

        if str(22) in (stops['ROUTE_LIST'][i]):
            for x in stops['ROUTE_LIST'][i].split(' '):
                if str(22) == x:
                    row = pd.DataFrame(stops.iloc[i])
                    stops_list = pd.concat([stops_list,row.transpose()])
                else:
                    pass
        else:
            pass
        
    stop_ids = stops_list['STOP_ID'].values
    
    geometry = stops_list.geometry.values

    xy = []
    for i in range(len(geometry)):
        dic = mapping(geometry[i])
        coords = dic['coordinates']
        xy.append(coords)
        xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
        xy_df['coordinates'] = xy
    
    xy_df['STOP_ID'] = stops_list['STOP_ID'].values
    
    df = pd.DataFrame(riders_num, columns = ['STOP_ID'])
    #df_ind = df.reset_index()
    #df_ind.columns = ['STOP_SEQ', 'STOP_ID', 'Mean']
    
    df_comb = xy_df.merge(df, how='right', on="STOP_ID")
    df_comb = df_comb.drop_duplicates(subset=['STOP_ID'], keep='first')
    df_comb = df_comb[['STOP_ID', 'coordinates']]
    #df_combine = df_comb.sort_values(by='STOP_SEQ')
    
    return xy_df, df_comb

In [ ]:
stop_coord(22, stop_ids)

In [ ]:
coordinates_45, final_45 = stop_coord(45, riders_45)

In [ ]:
coordinates_45.head()

In [ ]:
final_45.head()

In [ ]:
df_45_ami, riders_45ami, mass_45ami = route_ridership('AM', 'I', 45)
df_45amo, riders_45amo, mass_45amo = route_ridership('AM', 'O', 45)

In [ ]:
df_45pmi, riders_45pmi, mass_45pmi = route_ridership('PM', 'I', 45)
df_45pmo, riders_45pmo, mass_45pmo = route_ridership('PM', 'O', 45)

In [ ]:
riders_45pmi.head()

In [ ]:
df_45ami, riders_45ami, mass_45ami = route_ridership('AM', 'I', 45)
df_45midi, riders_45midi, mass_45midi = route_ridership('MID', 'I', 45)
df_45pmi, riders_45pmi, mass_45pmi = route_ridership('PM', 'I', 45)
df_45xnti, riders_45xnti, mass_45xnti = route_ridership('XNT', 'I', 45)
df_45xevi, riders_45xevi, mass_45xevi = route_ridership('XEV', 'I', 45)

In [ ]:
riders45ami = riders_45ami / 80
riders45midi = riders_45midi / 80
riders45pmi = riders_45pmi / 80
riders45xevi = riders_45xevi / 80
riders45xnti = riders_45xnti / 80

In [ ]:
riders45pmo = riders_45pmo / 80
riders45pmi = riders_45pmi / 80

In [ ]:
riders_45ami.head()

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(riders45pmo['Mean'], label='PM Outbound Segment', linewidth=4)
plt.plot(riders45pmi['Mean'], label='PM Inbound Segment', linewidth=4)
plt.legend()
plt.xlabel('Stop Sequence', size=16)
plt.ylabel('Average Number of Riders', size=16)
plt.title('Route 45 Inbound vs Outbound Riders', size=24)
plt.savefig('Outbound_Inbound_45.png', dpi=300)

In [ ]:
plt.figure(figsize=(8,8))
AM_plot = plt.plot(riders45ami['Mean'], label='Morning (5AM-9AM)', linewidth=4)
MID_plot = plt.plot(riders45midi['Mean'], label='Mid-Day (9AM-3PM)', linewidth=4)
PM_plot = plt.plot(riders45pmi['Mean'], label='Late Afternoon (3PM-7PM)', linewidth=4)
XEV_plot = plt.plot(riders45xevi['Mean'], label='Evening (7PM-10PM)', linewidth=4)
XNT_plot = plt.plot(riders45xnti['Mean'], label='Late Night (10PM-5AM)', linewidth=4)
plt.legend()
plt.xlabel('Stop Sequence', size=16)
plt.ylabel('Average Number of Riders', size=16)
plt.title('Number of Riders on Route 45 (Inbound)', size=24)
#plt.savefig('Ridership_daysegments.png', dpi=300)

In [ ]:
df_45amo, riders_45amo, mass_45amo = route_ridership('AM', 'O', 45)
df_45mido, riders_45mido, mass_45mido = route_ridership('MID', 'O', 45)
df_45pmo, riders_45pmo, mass_45pmo = route_ridership('PM', 'O', 45)
df_45xevo, riders_45xevo, mass_45xevo = route_ridership('XEV', 'O', 45)
df_45xnto, riders_45xnto, mass_45xnto = route_ridership('XNT', 'O', 45)

In [ ]:
AM_ploto = plt.plot(riders_45amo['Mean'], label='AM')
MID_ploto = plt.plot(riders_45mido['Mean'], label='MID')
PM_ploto = plt.plot(riders_45pmo['Mean'], label='PM')
XEV_ploto = plt.plot(riders_45xevo['Mean'], label='XEV')
XNT_ploto = plt.plot(riders_45xnto['Mean'], label='XNT')
plt.legend()